### Convert CSV into SQLITE Database

#### Data-Set
For this assignment purpose, the dataset will be used from [Kaggle Competition](https://www.kaggle.com/c/avazu-ctr-prediction/). Mostlikely, the same challenge will be capstone project as well.

#### Database
The training data base is access of **6Gb** when unzipped. To start with this for exercise, the data in csv will be converted into sqlite database and sql queries will be used for exploration purpose

In [3]:
import pandas as pd

df = pd.read_csv('../capstone_project/data/train.csv', nrows=2)
print(df.columns)
df.head()
                         

Index(['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object')


,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1000009418151094273,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15706,320,50,1722,0,35,-1,79
1,10000169349117863715,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79


### Converting csv file into sqllite database
Note: The actual function call is commented out so that this function does not get called all the time.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine # database connection
import datetime as dt
import os

def convert_csv_to_sqldb(csv_data, sql_data=None):
    start = dt.datetime.now()
    chunksize = 20000
    j = 0
    index_start = 1
    
    if sql_data is None:
        abs_path = os.path.dirname(os.path.abspath(csv_data))
        db_name = os.path.basename(csv_data).split('.')[0] + '.db'
        sql_path = os.path.join(abs_path, db_name)
        if os.path.exists(sql_path):
            os.remove(sql_path)
        sql_path = 'sqlite:///' + sql_path
    else:
        sql_path = sql_data
    
    # Initializes database
    disk_engine = create_engine(sql_path)
    
    for df in pd.read_csv(csv_data, chunksize=chunksize, iterator=True, encoding='utf-8'):
    
        df.hour = df.hour.astype(str)
        df.hour = df.hour.str[:2] + '-' + df.hour.str[2:]
        df.hour = df.hour.str[:5] + '-' + df.hour.str[5:]
        df.hour = df.hour.str[:8] + ' ' + df.hour.str[8:]
        df.hour = pd.to_datetime(df.hour, format='%y-%m-%d %H')

        df.index += index_start
    
        j+=1
        print('{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize))

        df.to_sql('data', disk_engine, if_exists='append')
        index_start = df.index[-1] + 1

convert_csv_to_sqldb('../capstone_project/data/train.csv')


0 seconds: completed 20000 rows
1 seconds: completed 40000 rows
2 seconds: completed 60000 rows
4 seconds: completed 80000 rows
5 seconds: completed 100000 rows
6 seconds: completed 120000 rows
7 seconds: completed 140000 rows
9 seconds: completed 160000 rows
10 seconds: completed 180000 rows
11 seconds: completed 200000 rows
12 seconds: completed 220000 rows
13 seconds: completed 240000 rows
14 seconds: completed 260000 rows
16 seconds: completed 280000 rows
17 seconds: completed 300000 rows
18 seconds: completed 320000 rows
19 seconds: completed 340000 rows
20 seconds: completed 360000 rows
21 seconds: completed 380000 rows
23 seconds: completed 400000 rows
24 seconds: completed 420000 rows
25 seconds: completed 440000 rows
26 seconds: completed 460000 rows
27 seconds: completed 480000 rows
28 seconds: completed 500000 rows
30 seconds: completed 520000 rows
31 seconds: completed 540000 rows
32 seconds: completed 560000 rows
33 seconds: completed 580000 rows
34 seconds: completed 6000

### Check Database Integrity

In [4]:
import os
import pandas as pd
from sqlalchemy import create_engine # database connection

def _get_sqldb_url(relative_path):
    abs_path = os.path.abspath(relative_path)
    return ('sqlite:///' + abs_path)
    
disk_engine = create_engine(_get_sqldb_url('../capstone_project/data/train.db'))    
db_df = pd.read_sql_query('SELECT * FROM data LIMIT 3', disk_engine)
print (db_df.columns)
db_df.head()


Index(['index', 'id', 'click', 'hour', 'C1', 'banner_pos', 'site_id',
       'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category',
       'device_id', 'device_ip', 'device_model', 'device_type',
       'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20',
       'C21'],
      dtype='object')


,index,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1,1000009418151094273,0,2014-10-21 00:00:00.000000,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,1,2,15706,320,50,1722,0,35,-1,79
1,2,10000169349117863715,0,2014-10-21 00:00:00.000000,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,1,0,15704,320,50,1722,0,35,100084,79
2,3,10000371904215119486,0,2014-10-21 00:00:00.000000,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,1,0,15704,320,50,1722,0,35,100084,79
